In [29]:
import json
from typing import Dict, List, Optional

import json
from typing import Dict, List, Set, Any, Optional


class JSONValidator:
    def __init__(self):
        self.errors = []
        self.warnings = []
        self.scope_symbols = {}
        self.all_scopes = []
        self.functions = {}
        self.source_map = {}  # Сопоставление узлов с исходными строками
        
    def validate(self, json_data: List[Dict]) -> Dict:
        """Основной метод валидации"""
        self.errors = []
        self.warnings = []
        self.scope_symbols = {}
        self.functions = {}
        self.all_scopes = json_data
        self.source_map = {}
        
        # Собираем информацию о всех узлах и их строках
        self.build_source_map(json_data)
        
        if not isinstance(json_data, list):
            self.add_error("JSON должен быть списком scope'ов")
            return self.get_report()
        
        # Собираем информацию о всех scope'ах и символах
        self.collect_symbols(json_data)  # <-- ЗДЕСЬ ВЫЗЫВАЕТСЯ
        
        # Проверяем каждый scope
        for scope_idx, scope in enumerate(json_data):
            self.validate_scope(scope, scope_idx, json_data)
        
        return self.get_report()
    
    def collect_symbols(self, json_data: List[Dict]):  # <-- ДОБАВИТЬ ЭТОТ МЕТОД
        """Собирает информацию о всех символах в системе"""
        for scope_idx, scope in enumerate(json_data):
            level = scope.get("level", 0)

            if "symbol_table" in scope and scope["symbol_table"]:
                
                if level not in self.scope_symbols:
                    self.scope_symbols[level] = {}
                
                for symbol_name, symbol_info in scope["symbol_table"].items():
                    # Сохраняем функции отдельно
                    if symbol_info.get("key") == "function":
                        self.functions[symbol_name] = symbol_info
                    else:
                        # Сохраняем обычные переменные
                        self.scope_symbols[level][symbol_name] = symbol_info
    
    def build_source_map(self, json_data: List[Dict]):
        """Строит карту соответствия узлов исходным строкам"""
        for scope_idx, scope in enumerate(json_data):
            level = scope.get("level", 0)
            graph = scope.get("graph", [])
            
            for node_idx, node in enumerate(graph):
                node_id = f"{scope_idx}.{node_idx}"
                # Сохраняем содержимое строки из поля 'content'
                self.source_map[node_id] = {
                    "content": node.get("content", ""),
                    "scope_idx": scope_idx,
                    "scope_level": level,
                    "scope_type": scope.get("type", "unknown")
                }
    
    def add_error(self, message: str, scope_idx: int = None, node_idx: int = None):
        """Добавляет ошибку с информацией о строке"""
        full_message = message
        
        if scope_idx is not None and node_idx is not None:
            node_id = f"{scope_idx}.{node_idx}"
            if node_id in self.source_map:
                content = self.source_map[node_id]["content"]
                if content:
                    full_message = f"Строка '{content}': {message}"
        
        self.errors.append(full_message)
    
    def add_warning(self, message: str, scope_idx: int = None, node_idx: int = None):
        """Добавляет предупреждение с информацией о строке"""
        full_message = message
        
        if scope_idx is not None and node_idx is not None:
            node_id = f"{scope_idx}.{node_idx}"
            if node_id in self.source_map:
                content = self.source_map[node_id]["content"]
                if content:
                    full_message = f"Строка '{content}': {message}"
        
        self.warnings.append(full_message)
    
    def validate_scope(self, scope: Dict, scope_idx: int, all_scopes: List[Dict]):
        """Валидирует отдельный scope"""
        level = scope.get("level", 0)
        scope_type = scope.get("type", "unknown")
        
        required_fields = ["level", "type", "local_variables", "graph", "symbol_table"]
        for field in required_fields:
            if field not in scope:
                self.add_error(f"Scope {scope_idx} (level {level}, type {scope_type}) отсутствует поле '{field}'", 
                             scope_idx, None)
        
        self.validate_symbol_table(scope, scope_idx)
        
        if "graph" in scope:
            self.validate_graph(scope, scope_idx)
        
        if scope_type == "function":
            self.validate_function_return(scope, scope_idx)
        
        self.validate_loops(scope, scope_idx)
    
    def validate_symbol_table(self, scope: Dict, scope_idx: int):
        """Валидирует таблицу символов scope'а"""
        symbol_table = scope.get("symbol_table", {})
        local_vars = scope.get("local_variables", [])
        
        for var_name in local_vars:
            if var_name not in symbol_table:
                self.add_warning(f"переменная '{var_name}' в local_variables отсутствует в symbol_table", 
                               scope_idx, None)
        
        for symbol_name, symbol_info in symbol_table.items():
            self.validate_symbol(symbol_info, scope_idx, symbol_name)
    
    def validate_symbol(self, symbol_info: Dict, scope_idx: int, symbol_name: str):
        """Валидирует отдельный символ"""
        required_fields = ["name", "key", "type", "id"]
        for field in required_fields:
            if field not in symbol_info:
                self.add_error(f"у символа '{symbol_name}' отсутствует поле '{field}'", 
                             scope_idx, None)
        
        if symbol_info.get("name") != symbol_name:
            self.add_warning(f"имя символа '{symbol_name}' не совпадает с полем 'name': {symbol_info.get('name')}", 
                           scope_idx, None)
        
        var_type = symbol_info.get("type", "")
        if var_type not in ["int", "string", "function", "None"] and not var_type.startswith("ptr_"):
            self.add_warning(f"символ '{symbol_name}' имеет неизвестный тип '{var_type}'", 
                           scope_idx, None)
        
        if symbol_info.get("key") == "const":
            if "value" not in symbol_info:
                self.add_error(f"константа '{symbol_name}' не имеет значения", 
                             scope_idx, None)
    
    def validate_graph(self, scope: Dict, scope_idx: int):
        """Валидирует граф операций"""
        graph = scope.get("graph", [])
        symbol_table = scope.get("symbol_table", {})
        level = scope.get("level", 0)
        
        for node_idx, node in enumerate(graph):
            node_type = node.get("node", "unknown")
            
            if node_type == "declaration":
                self.validate_declaration(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "assignment":
                self.validate_assignment(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "delete":
                self.validate_delete(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "augmented_assignment":
                self.validate_augmented_assignment(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "function_declaration":
                self.validate_function_declaration(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type in ["function_call", "function_call_assignment"]:
                self.validate_function_call(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "return":
                self.validate_return(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type in ["while_loop", "for_loop"]:
                self.validate_loop_node(node, node_idx, scope_idx, symbol_table, level)
            
            operations = node.get("operations", [])
            for op in operations:
                if op.get("type") == "UNARY_OPERATION":
                    self.validate_unary_operation(op, node_idx, scope_idx, level)
    
    def validate_declaration(self, node: Dict, node_idx: int, scope_idx: int, 
                           symbol_table: Dict, level: int):
        """Валидирует объявление переменной"""
        symbols = node.get("symbols", [])
        operations = node.get("operations", [])
        
        for symbol in symbols:
            if symbol not in symbol_table:
                self.add_error(f"объявляемая переменная '{symbol}' отсутствует в symbol_table", 
                             scope_idx, node_idx)
            else:
                for op in operations:
                    if op.get("type") in ["NEW_VAR", "NEW_CONST"]:
                        declared_type = op.get("var_type") or op.get("const_type")
                        actual_type = symbol_table[symbol].get("type")
                        if declared_type != actual_type:
                            self.add_error(f"тип переменной '{symbol}' не совпадает (объявлен: {declared_type}, в symbol_table: {actual_type})", 
                                         scope_idx, node_idx)
    
    def validate_assignment(self, node: Dict, node_idx: int, scope_idx: int, 
                      symbol_table: Dict, level: int):
        """Валидирует присваивание"""
        symbols = node.get("symbols", [])
        dependencies = node.get("dependencies", [])
        operations = node.get("operations", [])
        
        for symbol in symbols:
            symbol_info = self.get_symbol_info(symbol, level)
            if not symbol_info:
                self.add_error(f"присваиваемая переменная '{symbol}' не объявлена", 
                             scope_idx, node_idx)
            else:
                if symbol_info.get("key") == "const":
                    self.add_error(f"попытка присваивания константе '{symbol}'", 
                                 scope_idx, node_idx)
        
        for dep in dependencies:
            found = self.find_symbol_in_scope(dep, level)
            if not found:
                self.add_error(f"используемая переменная '{dep}' не объявлена", 
                             scope_idx, node_idx)
        
        for op in operations:
            if op.get("type") == "BINARY_OPERATION":
                left = op.get("left")
                right = op.get("right")
                
                for operand in [left, right]:
                    if operand and operand.isalpha() and operand not in ["True", "False", "None"]:
                        found = self.find_symbol_in_scope(operand, level)
                        if not found:
                            self.add_error(f"операнд '{operand}' не объявлен", 
                                         scope_idx, node_idx)
    
    def validate_delete(self, node: Dict, node_idx: int, scope_idx: int, 
                    symbol_table: Dict, level: int):
        """Валидирует удаление переменной"""
        symbols = node.get("symbols", [])
        
        for symbol in symbols:
            symbol_info = self.get_symbol_info(symbol, level)
            if not symbol_info:
                self.add_error(f"удаляемая переменная '{symbol}' не объявлена", 
                             scope_idx, node_idx)
            else:
                if symbol_info.get("key") == "const":
                    self.add_error(f"попытка удаления константы '{symbol}'", 
                                 scope_idx, node_idx)
    
    def validate_unary_operation(self, op: Dict, node_idx: int, scope_idx: int, level: int):
        """Валидирует унарную операцию"""
        value = op.get("value")
        
        if value and value.isalpha() and value not in ["True", "False", "None"]:
            if not self.find_symbol_in_scope(value, level):
                self.add_error(f"операнд унарной операции '{value}' не объявлен", 
                             scope_idx, node_idx)
    
    def validate_augmented_assignment(self, node: Dict, node_idx: int, scope_idx: int, 
                                  symbol_table: Dict, level: int):
        """Валидирует составное присваивание"""
        symbols = node.get("symbols", [])
        operations = node.get("operations", [])
        dependencies = node.get("dependencies", [])
        
        for symbol in symbols:
            symbol_info = self.get_symbol_info(symbol, level)
            if not symbol_info:
                self.add_error(f"переменная '{symbol}' в составном присваивании не объявлена", 
                             scope_idx, node_idx)
            else:
                if symbol_info.get("key") == "const":
                    self.add_error(f"попытка модификации константы '{symbol}'", 
                                 scope_idx, node_idx)
        
        for dep in dependencies:
            if not self.find_symbol_in_scope(dep, level):
                self.add_error(f"используемая переменная '{dep}' не объявлена", 
                             scope_idx, node_idx)
    
    def validate_function_declaration(self, node: Dict, node_idx: int, scope_idx: int, 
                                  symbol_table: Dict, level: int):
        """Валидирует объявление функции"""
        func_name = node.get("function_name")
        parameters = node.get("parameters", [])
        
        for other_scope in self.all_scopes:
            if other_scope.get("level") <= level:
                for other_node in other_scope.get("graph", []):
                    if (other_node.get("node") == "function_declaration" and 
                        other_node.get("function_name") == func_name and 
                        other_node is not node):
                        self.add_error(f"функция '{func_name}' уже объявлена", 
                                     scope_idx, node_idx)
                        return
        
        param_names = set()
        for param in parameters:
            param_name = param.get("name")
            if param_name in param_names:
                self.add_error(f"дублирующийся параметр '{param_name}' в функции '{func_name}'", 
                             scope_idx, node_idx)
            param_names.add(param_name)
    
    def validate_function_call(self, node: Dict, node_idx: int, scope_idx: int, 
                             symbol_table: Dict, level: int):
        """Валидирует вызов функции"""
        func_name = node.get("function")
        arguments = node.get("arguments", [])
        
        if func_name not in self.functions:
            self.add_error(f"вызываемая функция '{func_name}' не объявлена", 
                         scope_idx, node_idx)
        else:
            func_info = self.functions[func_name]
            func_params = func_info.get("parameters", [])
            
            if len(arguments) != len(func_params):
                self.add_error(f"функция '{func_name}' ожидает {len(func_params)} аргументов, передано {len(arguments)}", 
                             scope_idx, node_idx)
        
        for arg in arguments:
            if arg and arg.isalpha() and arg not in ["True", "False", "None"]:
                if not self.find_symbol_in_scope(arg, level):
                    self.add_error(f"аргумент '{arg}' не объявлен", 
                                 scope_idx, node_idx)
    
    def validate_return(self, node: Dict, node_idx: int, scope_idx: int, 
                       symbol_table: Dict, level: int):
        """Валидирует оператор return"""
        dependencies = node.get("dependencies", [])
        
        for dep in dependencies:
            if not self.find_symbol_in_scope(dep, level):
                self.add_error(f"возвращаемая переменная '{dep}' не объявлена", 
                             scope_idx, node_idx)
    
    def validate_loop_node(self, node: Dict, node_idx: int, scope_idx: int, 
                          symbol_table: Dict, level: int):
        """Валидирует узел цикла"""
        node_type = node.get("node")
        
        if node_type == "while_loop":
            condition = node.get("condition", {})
            if condition.get("type") == "COMPARISON":
                left = condition.get("left")
                right = condition.get("right")
                
                for var in [left, right]:
                    if var and var.isalpha() and var not in ["True", "False", "None"]:
                        if not self.find_symbol_in_scope(var, level):
                            self.add_error(f"переменная '{var}' в условии цикла не объявлена", 
                                         scope_idx, node_idx)
        
        elif node_type == "for_loop":
            loop_var = node.get("loop_variable")
            iterable = node.get("iterable", {})
            
            if loop_var not in symbol_table:
                self.add_error(f"переменная цикла '{loop_var}' не объявлена", 
                             scope_idx, node_idx)
            
            if iterable.get("type") == "RANGE_CALL":
                args = iterable.get("arguments", {})
                for arg_name, arg_value in args.items():
                    if arg_value and arg_value.isalpha() and arg_value not in ["True", "False", "None"]:
                        if not self.find_symbol_in_scope(arg_value, level):
                            self.add_error(f"аргумент range '{arg_value}' не объявлен", 
                                         scope_idx, node_idx)
    
    def validate_function_return(self, scope: Dict, scope_idx: int):
        """Проверяет, что функция имеет return если нужно"""
        return_info = scope.get("return_info", {})
        return_type = scope.get("return_type", "None")
        
        if return_type != "None" and not return_info.get("has_return", False):
            # Находим строку с объявлением функции
            func_content = ""
            for node_idx, node in enumerate(scope.get("graph", [])):
                if node.get("node") == "function_declaration":
                    func_content = node.get("content", "")
                    break
            
            if func_content:
                self.add_warning(f"функция возвращает '{return_type}' но не имеет оператора return", 
                               scope_idx, None)
    
    def validate_loops(self, scope: Dict, scope_idx: int):
        """Проверяет циклы на корректность"""
        graph = scope.get("graph", [])
        
        for node_idx, node in enumerate(graph):
            if node.get("node") in ["while_loop", "for_loop"]:
                body = node.get("body", [])
                if not body:
                    self.add_warning(f"тело цикла пустое", scope_idx, node_idx)
    
    def find_symbol_in_scope(self, symbol_name: str, current_level: int) -> bool:
        """Ищет символ в текущем или родительских scope'ах"""
        if current_level in self.scope_symbols and symbol_name in self.scope_symbols[current_level]:
            return True
        
        current_scope = None
        for scope in self.all_scopes:
            if scope.get("level") == current_level:
                current_scope = scope
                break
        
        if current_scope:
            parent_level = current_scope.get("parent_scope")
            if parent_level is not None:
                return self.find_symbol_in_scope(symbol_name, parent_level)
        
        return False
    
    def get_symbol_info(self, symbol_name: str, current_level: int) -> Optional[Dict]:
        """Получает информацию о символе из текущего или родительских scope'ов"""
        if (current_level in self.scope_symbols and 
            symbol_name in self.scope_symbols[current_level]):
            return self.scope_symbols[current_level][symbol_name]
        
        for level, symbols in self.scope_symbols.items():
            if symbol_name in symbols:
                return symbols[symbol_name]
        
        return None
    
    def get_report(self) -> Dict:
        """Возвращает отчет о проверке"""
        return {
            "is_valid": len(self.errors) == 0,
            "error_count": len(self.errors),
            "warning_count": len(self.warnings),
            "errors": self.errors,
            "warnings": self.warnings
        }

# Пример использования
def validate_json_file(json_file_path: str):
    """Валидирует JSON файл"""
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
    except Exception as e:
        return {
            "is_valid": False,
            "error_count": 1,
            "warning_count": 0,
            "errors": [f"Ошибка загрузки JSON: {str(e)}"],
            "warnings": []
        }
    
    validator = JSONValidator()
    result = validator.validate(json_data)
    
    return result


# Тестовый пример
if __name__ == "__main__":
    path = "/Users/phil/GitHub/phils_language/parsed_code.json"

    with open(path, "r") as file:
        data = json.load(file)
    
    validator = JSONValidator()
    result = validator.validate(data)
    
    print("Результат валидации:")
    print(f"Валидный: {result['is_valid']}")
    print(f"Ошибок: {result['error_count']}")
    print(f"Предупреждений: {result['warning_count']}")
    
    if result['errors']:
        print("\nОшибки:")
        for error in result['errors']:
            print(f"  - {error}")
    
    if result['warnings']:
        print("\nПредупреждения:")
        for warning in result['warnings']:
            print(f"  - {warning}")

Результат валидации:
Валидный: False
Ошибок: 2
Предупреждений: 0

Ошибки:
  - Строка 'sum = sum + dd': используемая переменная 'dd' не объявлена
  - Строка 'sum = sum + dd': операнд 'dd' не объявлен
